In [2]:
%load_ext autoreload
%autoreload 2
import requests
from bs4 import BeautifulSoup
import pandas as pd
from lxml import etree
import datetime

In [3]:
pd.read_csv('final_data_async.csv')

,Unnamed: 0,carros,price,link,vehicle_info
0,0,Toyota 4runner 4.0 Sr5,315000000,https://carro.mercadolibre.com.co/MCO-21498814...,"{'Year': '2022', 'Kilometrage': '25720 ', 'Pub..."
1,1,Renault Captur Intens,112000000,https://carro.mercadolibre.com.co/MCO-21496291...,"{'Year': '2022', 'Kilometrage': '26350 ', 'Pub..."
2,2,Chevrolet Sonic Lt,42000000,https://carro.mercadolibre.com.co/MCO-21496930...,"{'Year': '2016', 'Kilometrage': '36000 ', 'Pub..."
3,3,Jeep Grand Cherokee 3.6 Limited,178000000,https://carro.mercadolibre.com.co/MCO-21496795...,"{'Year': '2020', 'Kilometrage': '24000 ', 'Pub..."
4,4,Nissan Note Sense,38000000,https://carro.mercadolibre.com.co/MCO-21497432...,"{'Year': '2015', 'Kilometrage': '92000 ', 'Pub..."
5,5,Peugeot 308 2.0 Premium,26000000,https://carro.mercadolibre.com.co/MCO-21496147...,"{'Year': '2009', 'Kilometrage': '127500 ', 'Pu..."
6,6,Honda Cr-v 2.4 Lx At,59900000,https://carro.mercadolibre.com.co/MCO-21498794...,"{'Year': '2011', 'Kilometrage': '89729 ', 'Pub..."
7,7,Chevrolet Sonic 1.6 Lt 4 P Automática,45000000,https://carro.mercadolibre.com.co/MCO-21496274...,"{'Year': '2017', 'Kilometrage': '77000 ', 'Pub..."
8,8,Toyota Hilux 2.8l,198000000,https://carro.mercadolibre.com.co/MCO-21496271...,"{'Year': '2020', 'Kilometrage': '50040 ', 'Pub..."
9,9,Ford Focus Titanium,38000000,https://carro.mercadolibre.com.co/MCO-21496912...,"{'Year': '2013', 'Kilometrage': '77946 ', 'Pub..."


In [18]:
def get_soup_by_url(url, product: str = None):
    if product is None:
        url = url
    else:
        url = f'{url}/{product}'
    r = requests.get(url=url)
    s = BeautifulSoup(r.content, 'html.parser')
    return s

def get_vehicle_info(url):
    s = get_soup_by_url(url)
    text = s.find_all('span', attrs= {'class':'ui-pdp-subtitle'})[0].text.replace('.', '')
    parts = text.split(' · ')
    location = s.find('p', attrs = {'class':'ui-seller-info__status-info__subtitle'}).text
    pub_number = s.find_all('span', attrs= {'class':'ui-pdp-color--BLACK ui-pdp-family--SEMIBOLD'})[0].text.replace('#','')
    year = parts[0].split(' | ')[0]
    kilometrage = parts[0].split(' | ')[1].replace('km', '')
    publication_date = parts[1]
    output_dict = {
        "Year": year,
        "Kilometrage": kilometrage,
        "Publication Date": publication_date,
        "Location": location,
        "Pub Number": pub_number,
        "Created At": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    }

    return output_dict
data = pd.read_csv('final_data.csv')

In [21]:
data['link'][:5].apply(get_vehicle_info)

0    {'Year': '2017', 'Kilometrage': '47800 ', 'Pub...
1    {'Year': '2015', 'Kilometrage': '72000 ', 'Pub...
2    {'Year': '2015', 'Kilometrage': '73251 ', 'Pub...
3    {'Year': '2012', 'Kilometrage': '75500 ', 'Pub...
4    {'Year': '2017', 'Kilometrage': '80000 ', 'Pub...
Name: link, dtype: object

In [19]:
data['link'].apply(get_vehicle_info)

KeyboardInterrupt: 

In [2]:
link = 'https://carro.mercadolibre.com.co/MCO-1379439831-mercedes-benz-gle-350-d-4matic-2017-_JM#position=1&search_layout=grid&type=item&tracking_id=79a64cb3-adcf-4d29-be31-812b12b34cef'

In [11]:
link

'https://carro.mercadolibre.com.co/MCO-1379439831-mercedes-benz-gle-350-d-4matic-2017-_JM#position=1&search_layout=grid&type=item&tracking_id=79a64cb3-adcf-4d29-be31-812b12b34cef'

In [3]:
r = requests.get(url=link)

s = BeautifulSoup(r.content, 'html.parser')

In [4]:
# publication number

int(s.find_all('span', attrs= {'class':'andes-money-amount__fraction'})[0].text.replace('.', ''))

189000000

In [5]:
# precio
s.find_all('span', attrs= {'class':'ui-pdp-color--BLACK ui-pdp-family--SEMIBOLD'})[0].text.replace('#','')

'1379439831'

In [16]:
import datetime

text = s.find_all('span', attrs= {'class':'ui-pdp-subtitle'})[0].text.replace('.', '')
parts = text.split(' · ')
location = s.find('p', attrs = {'class':'ui-seller-info__status-info__subtitle'}).text
pub_number = s.find_all('span', attrs= {'class':'ui-pdp-color--BLACK ui-pdp-family--SEMIBOLD'})[0].text.replace('#','')
year = parts[0].split(' | ')[0]
kilometrage = parts[0].split(' | ')[1].replace('km', '')
publication_date = parts[1]
output_dict = {
  "Year": year,
  "Kilometrage": kilometrage,
  "Publication Date": publication_date,
  "Location": location,
  "Pub Number": pub_number,
  "Created At": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
}

output_dict

{'Year': '2017',
 'Kilometrage': '47800 ',
 'Publication Date': 'Publicado hace 3 días',
 'Location': 'Chicó Navarra - Usaquén - Bogotá D.C.',
 'Pub Number': '1379439831',
 'Created At': '2024-01-13 13:35:22'}

'Chicó Navarra - Usaquén - Bogotá D.C.'

In [8]:
table = s.find_all('table', attrs= {'class': 'andes-table'})

In [9]:
table

[]

In [10]:
rows = table[0].find_all('tr')
rows

IndexError: list index out of range

In [106]:
data = {}
for row in rows:
  cols = row.find_all(['th', 'td'])
  cols = [ele.text.strip() for ele in cols]
  data[cols[0]] = cols[1]


In [107]:
data

{'Marca': 'Mercedes-Benz',
 'Modelo': 'Clase GLE',
 'Año': '2017',
 'Versión': '3.0 Coupe 4matic Diésel',
 'Color': 'Gris',
 'Tipo de combustible': 'Diésel',
 'Puertas': '5',
 'Transmisión': 'Automática',
 'Motor': '3.0',
 'Tipo de carrocería': 'Station Wagon',
 'Kilómetros': '47800 km'}

In [22]:
rows

NameError: name 'rows' is not defined

In [49]:
dom = etree.HTML(str(s))

In [50]:
dom

<Element html at 0x7baa0c6bbfc0>

In [51]:
dom.xpath("//table[@class = 'andes-table']")

[]